# Blablattractivity - Local and tests -

## Data collection

We start with basic imports to use blablacar web api.

In [1]:
#%pip install pandas

In [2]:
#%pip install fsspec

In [3]:
#%pip install google-cloud-storage

In [17]:
#%pip install matplotlib

In [18]:
import json
import requests
import pandas as pd
from google.cloud import storage
import datetime 
import matplotlib.pyplot as plt

Dès qu'on reçoit l'API on part là-dessus : https://dev.blablacar.com/hc/en-us/articles/360012064820--Search-V3-API-Documentation

In [5]:
api_token = open("./api.txt","r").read()
api_url = "https://public-api.blablacar.com"

We have the basic parameters of our API call :

In [6]:
from_coordinate = "48.117266%2C-1.677793" # string : latitude, longitude - RENNES
to_coordinate = "48.390394%2C-4.486076" # string : latitude, longitude - BREST
from_country = "FR" # string : ISO_3166_1
to_country = "FR" # string : ISO_3166_1
locale = "fr-FR" # string : locale in which the result will be displayed https://dev.blablacar.com/hc/en-us/articles/360008930020
currency = "EUR" # string : ISO_4217 currency
number_of_trips_we_want_returned = "100" # int : from 1 to 100, default is 10 
requested_seats = "1" # int : minimum number of seats requested, default value is 1
radius_in_meters = "6000" # int : radius of the circle around the coordinates in which trips will be searched from (in meters)

Then the date of the trip will be different everytime we run the code (it has to be next friday everytime), so we compute it here :

In [7]:
today = datetime.date.today()
friday = today + datetime.timedelta( (4-today.weekday()) % 7 )
next_friday_datetime = str(friday)+"T00:00:00"
start_date_local = next_friday_datetime

In [8]:
response = requests.get(api_url+"/api/v3/trips?key="+api_token+"&from_coordinate="+from_coordinate+"&from_country="
                        +from_country+"&to_coordinate="+to_coordinate+"&to_country="+to_country+"&locale="+locale+"&currency="
                        +currency+"&count="+number_of_trips_we_want_returned+"&start_date_local="+start_date_local
                        +"&radius_in_meters="+radius_in_meters+"&sort=price:asc") 

In [9]:
print(response.status_code) # SHOULD BE 200

200


In [10]:
json_trips = response.json() # it's a dictionnary

In [11]:
new_dict = {'trip_id':[],'price_of_the_trip':[],'date':[]}
for dict_trip in json_trips["trips"]:
    trip_id =  dict_trip["link"].split("id=")[1].split("-")[0]
    price_of_the_trip = dict_trip["price"]["amount"]
    new_dict['trip_id'].append(trip_id)
    new_dict['price_of_the_trip'].append(price_of_the_trip)
    new_dict['date'].append(start_date_local[:10])

In [29]:
new_df = pd.DataFrame.from_dict(new_dict)
new_df.price_of_the_trip = new_df.price_of_the_trip.astype('float')    

trip_id               object
price_of_the_trip    float64
date                  object
dtype: object

In [13]:
#old_df = pd.read_csv("./blablacar_rennes_brest.csv",index_col = "Unnamed: 0") 
#old_df.head()
#df_trips = old_df.append(new_df, ignore_index=True, sort=False)
#df_trips.head()
#df_trips.to_csv("./blablacar_rennes_brest.csv")

## Visualization

In [35]:
df_trips = pd.read_csv("./blablacar_rennes_brest.csv",index_col = "Unnamed: 0") 

### Plot number of trips per day

In [36]:
df_count = df_trips.groupby(df_trips['date']).size().reset_index(name='count')
df_count

,date,count
0,2020-08-21,5
1,2020-08-28,6


In [40]:
df_trips

,trip_id,price_of_the_trip,date
0,2030433816,11.25,2020-08-21
1,2026019926,12.50,2020-08-21
2,2014886656,12.50,2020-08-21
3,2030108726,12.50,2020-08-21
4,2029796031,13.75,2020-08-21
5,2030436816,14.25,2020-08-28
6,2026016926,12.50,2020-08-28
7,2014888656,12.50,2020-08-28
8,2030106726,12.00,2020-08-28
9,2029798031,13.75,2020-08-28


In [14]:
plt.plot()
new_df

,trip_id,price_of_the_trip,date
0,2030433816,11.25,2020-08-21
1,2030837206,12.50,2020-08-21
2,2026019926,12.50,2020-08-21
3,2032368386,12.50,2020-08-21
4,2032398641,13.75,2020-08-21
5,2032315926,13.75,2020-08-21
6,2031945171,13.75,2020-08-21
7,2030159916,17.50,2020-08-21
8,2032511266,18.75,2020-08-21


### Plot mean price per day

In [38]:
df_mean = df_trips.groupby(['date'])['price_of_the_trip'].mean().reset_index(name='mean')
df_mean

,date,mean
0,2020-08-21,12.500000
1,2020-08-28,13.458333


### Plot median price per day

In [39]:
df_median = df_trips.groupby(['date'])['price_of_the_trip'].median().reset_index(name='median')
df_median

,date,median
0,2020-08-21,12.500
1,2020-08-28,13.125


### Frequency plot

In [42]:
df_price_freq = df_trips.groupby(df_trips['price_of_the_trip']).size().reset_index(name='count')
df_price_freq

,price_of_the_trip,count
0,11.25,1
1,12.00,1
2,12.50,5
3,13.75,2
4,14.25,1
5,15.75,1
